In [1]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

In [2]:
model = YOLO('https://github.com/akanametov/yolov8-face/releases/download/v0.0.0/yolov8l-face.pt')
names = model.names

Found https://github.com/akanametov/yolov8-face/releases/download/v0.0.0/yolov8l-face.pt locally at weights\yolov8l-face.pt


In [4]:
def croped_image(input_image_dir, destination):
    # Desired size for the cropped images
    desired_size = (190, 250)
    # Padding factor to enlarge the bounding box
    padding_factor = 0.1  # 10% padding

    # Iterate over each subdirectory in the input directory
    for sub_dir in os.listdir(input_image_dir):
        sub_dir_path = os.path.join(input_image_dir, sub_dir)

        # Create corresponding output directory
        output_dir = os.path.join(destination, sub_dir)
        os.makedirs(output_dir, exist_ok=True)

        # Process each image in the subdirectory
        for image_file in os.listdir(sub_dir_path):
            image_path = os.path.join(sub_dir_path, image_file)
            im0 = cv2.imread(image_path)

            if im0 is None:
                print(f"Error loading image {image_path}")
                continue

            results = model.predict(im0, show=False)
            boxes = results[0].boxes.xyxy.cpu().tolist()
            clss = results[0].boxes.cls.cpu().tolist()

            if boxes is not None:
                idx = 0
                for box, cls in zip(boxes, clss):
                    # Check if the detected class is a face (assuming class 0 is 'face')
                    if model.names[int(cls)] == 'face':
                        idx += 1

                        # Calculate padding
                        width = box[2] - box[0]
                        height = box[3] - box[1]
                        pad_w = width * padding_factor
                        pad_h = height * padding_factor

                        # Adjust the bounding box with padding
                        x1 = max(0, int(box[0] - pad_w))
                        y1 = max(0, int(box[1] - pad_h))
                        x2 = min(im0.shape[1], int(box[2] + pad_w))
                        y2 = min(im0.shape[0], int(box[3] + pad_h))

                        # Crop the object (face) from the image
                        crop_obj = im0[y1:y2, x1:x2]

                        # Resize the cropped object to the desired size
                        crop_obj_resized = cv2.resize(crop_obj, desired_size)

                        # Save the resized cropped face image
                        crop_file_path = os.path.join(output_dir, f"{os.path.splitext(image_file)[0]}_{idx}.png")
                        cv2.imwrite(crop_file_path, crop_obj_resized)
                    else:
                        print(f"Skipping non-face object in {image_file}")

In [7]:
croped_image('training_set', 'train_(190x250)')


0: 960x800 1 face, 1199.3ms
Speed: 11.0ms preprocess, 1199.3ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 800)

0: 960x768 1 face, 1154.2ms
Speed: 6.0ms preprocess, 1154.2ms inference, 1.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x800 1 face, 1168.8ms
Speed: 6.0ms preprocess, 1168.8ms inference, 1.5ms postprocess per image at shape (1, 3, 960, 800)

0: 960x928 1 face, 1389.1ms
Speed: 8.0ms preprocess, 1389.1ms inference, 1.1ms postprocess per image at shape (1, 3, 960, 928)

0: 960x704 1 face, 1044.9ms
Speed: 5.8ms preprocess, 1044.9ms inference, 0.0ms postprocess per image at shape (1, 3, 960, 704)

0: 960x768 1 face, 1132.3ms
Speed: 4.2ms preprocess, 1132.3ms inference, 1.9ms postprocess per image at shape (1, 3, 960, 768)

0: 960x768 1 face, 1078.8ms
Speed: 4.8ms preprocess, 1078.8ms inference, 0.0ms postprocess per image at shape (1, 3, 960, 768)

0: 960x736 1 face, 1129.1ms
Speed: 5.0ms preprocess, 1129.1ms inference, 0.5ms postprocess per image a